## In this notebook:

* on feature set 4

### Imports

In [1]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [2]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset4.pkl")
df = df.replace({'-': 0})

# problematic isn't first:

#print("LENGTH",len(df.columns))
col = df.pop("Problematic")
df[col.name] = col

# Split our data into three separate sets for training purposes
train, test = train_test_split(df, shuffle=False)
train, valid = train_test_split(train, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                       aapos codonpos  Ancestral_allele  Polyphen2_HVAR_pred  \
chr pos       ref alt                                                          
3   101244651 A   T      560        3                 2                    0   
    101244766 G   C      522        2                 2                    0   
    101269574 C   A       -1        0                 2                    0   
    101275694 G   C      212        2                 2                    0   
    101291499 A   C      171        3                 2                    0   

                          GenoCanyon_score HUVEC_fitCons_score  \
chr pos       ref alt                                            
3   101244651 A   T      0.995787957382137            0.567892   
    101244766 G   C      0.997773309808223            0.567892   
    101269574 C   A      0.998404006175971            0.057018   
    101275694 G   C    1.97468820369911E-4            0.620846   
    101291499 A   C      0.985239277427167 

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [3]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(valid)  
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [4]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(14, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 14))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [5]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [0/7773 (0%)]	Loss: 0.699625
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [59/7773 (1%)]	Loss: 0.811362
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [118/7773 (2%)]	Loss: 0.737815
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [177/7773 (2%)]	Loss: 0.627957
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [236/7773 (3%)]	Loss: 0.675640
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [295/7773 (4%)]	Loss: 0.534842
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [354/7773 (5%)]	Loss: 0.557231
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [413/7773 (5%)]	Loss: 0.580874
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [472/7773 (6%)]	Loss: 0.653909
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [531/7773 (7%)]	Loss: 0.691082
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [590/7773 (8%)]	Loss: 0.588299
torch.Size([59, 14])
torch.Size([59

 Train Epoch: 1/20 [5369/7773 (69%)]	Loss: 0.634857
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5428/7773 (70%)]	Loss: 0.584459
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5487/7773 (70%)]	Loss: 0.652580
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5546/7773 (71%)]	Loss: 0.583630
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5605/7773 (72%)]	Loss: 0.700384
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5664/7773 (73%)]	Loss: 0.550429
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5723/7773 (73%)]	Loss: 0.635921
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5782/7773 (74%)]	Loss: 0.637238
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5841/7773 (75%)]	Loss: 0.604761
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5900/7773 (76%)]	Loss: 0.652414
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5959/7773 (77%)]	Loss: 0.698169
torch.Size([59, 14])
torch.Size([59])
 Train Epo

 Train Epoch: 2/20 [2655/7773 (34%)]	Loss: 0.669280
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2714/7773 (35%)]	Loss: 0.567585
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2773/7773 (36%)]	Loss: 0.746372
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2832/7773 (36%)]	Loss: 0.680020
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2891/7773 (37%)]	Loss: 0.735282
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [2950/7773 (38%)]	Loss: 0.716758
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3009/7773 (39%)]	Loss: 0.674485
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3068/7773 (39%)]	Loss: 0.583966
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3127/7773 (40%)]	Loss: 0.616748
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3186/7773 (41%)]	Loss: 0.593754
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [3245/7773 (42%)]	Loss: 0.522932
torch.Size([59, 14])
torch.Size([59])
 Train Epo

 Train Epoch: 3/20 [0/7773 (0%)]	Loss: 0.605191
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [59/7773 (1%)]	Loss: 0.567275
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [118/7773 (2%)]	Loss: 0.616461
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [177/7773 (2%)]	Loss: 0.700447
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [236/7773 (3%)]	Loss: 0.537830
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [295/7773 (4%)]	Loss: 0.586343
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [354/7773 (5%)]	Loss: 0.553441
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [413/7773 (5%)]	Loss: 0.641752
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [472/7773 (6%)]	Loss: 0.663331
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [531/7773 (7%)]	Loss: 0.736151
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [590/7773 (8%)]	Loss: 0.620085
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [649/7773 (8%)]	

 Train Epoch: 3/20 [5428/7773 (70%)]	Loss: 0.700302
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5487/7773 (70%)]	Loss: 0.516914
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5546/7773 (71%)]	Loss: 0.534093
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5605/7773 (72%)]	Loss: 0.596066
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5664/7773 (73%)]	Loss: 0.582500
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5723/7773 (73%)]	Loss: 0.546966
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5782/7773 (74%)]	Loss: 0.617941
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5841/7773 (75%)]	Loss: 0.583239
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5900/7773 (76%)]	Loss: 0.646649
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [5959/7773 (77%)]	Loss: 0.577789
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [6018/7773 (77%)]	Loss: 0.634307
torch.Size([59, 14])
torch.Size([59])
 Train Epo

...

 Train Epoch: 18/20 [2419/7773 (31%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2478/7773 (32%)]	Loss: 0.652245
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2537/7773 (33%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2596/7773 (33%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2655/7773 (34%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2714/7773 (35%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2773/7773 (36%)]	Loss: 0.567499
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2832/7773 (36%)]	Loss: 0.669194
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2891/7773 (37%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [2950/7773 (38%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 18/20 [3009/7773 (39%)]	Loss: 0.635296
torch.Size([59, 14])
torch.Size([59])

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([1., 1., 1.,  ..., 1., 1., 0.])
 Train Epoch: 18/20 [7773/7773 (99%)]	Loss: 0.563262	 Validation Accuracy: 76.77%
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [0/7773 (0%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [59/7773 (1%)]	Loss: 0.669194
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [118/7773 (2%)]	Loss: 0.736991
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [177/7773 (2%)]	Loss: 0.652245
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [236/7773 (3%)]	Loss: 0.499702
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [295/7773 (4%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [354/7773 (5%)]	Loss: 0.516651
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [413/7773 (5%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [472/7773 (6%)]	Loss: 0.567499
torch.Size([59, 14])

 Train Epoch: 19/20 [5192/7773 (67%)]	Loss: 0.652245
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5251/7773 (67%)]	Loss: 0.499702
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5310/7773 (68%)]	Loss: 0.567499
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5369/7773 (69%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5428/7773 (70%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5487/7773 (70%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5546/7773 (71%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5605/7773 (72%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5664/7773 (73%)]	Loss: 0.669194
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5723/7773 (73%)]	Loss: 0.736991
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 19/20 [5782/7773 (74%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 20/20 [2419/7773 (31%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2478/7773 (32%)]	Loss: 0.567499
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2537/7773 (33%)]	Loss: 0.669194
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2596/7773 (33%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2655/7773 (34%)]	Loss: 0.652245
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2714/7773 (35%)]	Loss: 0.686143
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2773/7773 (36%)]	Loss: 0.652245
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2832/7773 (36%)]	Loss: 0.736991
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2891/7773 (37%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [2950/7773 (38%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 20/20 [3009/7773 (39%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([1., 1., 1.,  ..., 1., 1., 0.])
 Train Epoch: 20/20 [7773/7773 (99%)]	Loss: 0.654171	 Validation Accuracy: 76.77%


### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [6]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]


y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))

print(len(pred))
print(len(y_labels))


 Testing Accuracy: 68.80%
3455
3455


### Metrics
* Use sklearn builtins to calculate different metrics

In [7]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

Confusion matrix: 
 [[   0 1078]
 [   0 2377]]
Accuracy: 0.6879884225759768
Recall TP/(TP+FN): [0. 1.]
Precision TP/(TP+FP): [0.         0.68798842]
F1: [       nan 0.81515775]


c:\users\vincent\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vincent\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


In [8]:
# Recalculating accuracy: TP + TN / (all exmaples)
print("accuracy", 2362/(1080+2362))
1080+2362

#Test shape, train shape
#(10325, 9) (3442, 9)

accuracy 0.6862289366647298


3442